In [7]:
from st_graph_generation import detect_with_networkx
from common_utils.stg_utils import get_video_params
from common_utils.process import graph_to_feature_vector

import torch
import time
import argparse
from networkx import adjacency_matrix

In [4]:
with torch.no_grad():
        #t = time.time()
        videoname_to_graph_seq = detect_with_networkx.detect(source="../videos/")
        #print(f'Done. ({time.time() - t:.3f}s)')

videoname_to_graph_seq

YOLOR 🚀 2022-11-24 torch 1.12.1 CPU



Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block


Model Summary: 306 layers, 36905341 parameters, 6652669 gradients


RepConv.fuse_repvgg_block

video 1/1 (1/132) half_street.mp4

/Users/niccolomorabito/Library/Python/3.9/lib/python/site-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/TensorShape.cpp:2895.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


video 1/1 (132/132) half_street.mp4Done. (55.281s)


{'half_street.mp4': [<networkx.classes.graph.Graph at 0x179d55490>,
  <networkx.classes.graph.Graph at 0x28f252910>]}

In [8]:
for videoname in videoname_to_graph_seq:
    print(len(videoname_to_graph_seq[videoname]))
    for graph in videoname_to_graph_seq[videoname]:
        #print(type(adjacency_matrix(graph)))
        break
a = graph_to_feature_vector(graph)
print(a.shape)

132
(22, 85)


# CORRUPTION
(see if it must be done during the process or it can be done as follows after all)

HOW CORRUPTUION CODE SHOULD BE CALLED

```
for videoclip:
    corruptor = Corruptor(..., ...)
    normal_sequence = list()
    corrupted_sequence = list()

    for frame in clip:
        graph = //graph initialization
        corrupted_graph = corruptor.corrupt_graph(graph)
        normal_sequence.append(graph)
        corrupted_sequence.append(corrupted_graph)
        // do we need object tracking etc.???
```

In [ ]:
from anomaly_generation.graph_corruption import Corruptor

videoname_to_corr_graph_seq = dict()
for videoname in videoname_to_graph_seq:
    video_params = get_video_params(f'../videos/{videoname}')
    w, h = video_params["width"], video_params["height"]
    #TODO so far, is_stg must be true because only the distance is implemented on the edge as 'weight'
    corruptor = Corruptor(frame_width=w, frame_height=h, is_stg=True)

    videoname_to_corr_graph_seq[videoname] = list()
    
    for graph in videoname_to_graph_seq[videoname]:
        corrupted_graph = corruptor.corrupt_graph(graph, verbose=False)
        videoname_to_corr_graph_seq[videoname].append(corrupted_graph)



In [ ]:

for videoname in videoname_to_graph_seq:
    graphs = videoname_to_graph_seq[videoname]
    corr_graphs = videoname_to_corr_graph_seq[videoname]
    break

graphs